In [ ]:
%cd /nfs/home/medoro/Unsupervised_Anomaly_Detection_thesis

In [ ]:
from preprocessing import *
import preprocessing as prp
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as data_utils
from usad import *
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, roc_auc_score
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataframe = pd.read_csv(r"/nfs/home/medoro/Unsupervised_Anomaly_Detection_thesis/data/train.csv")
dataframe.shape

In [ ]:
df=dataframe[['building_id','primary_use', 'timestamp', 'meter_reading', 'sea_level_pressure', 'is_holiday','anomaly']]
df

In [ ]:
imputed_df = impute_nulls(df)
imputed_df

Now that we have imputed the missing values for the column containing the energy consumption measurements, we can procees by adding a couple of features more and further imputing the missing dates for each timeseries in the dataset.

In [ ]:
df = add_trigonometric_features(imputed_df)
df

In [ ]:
dfs_dict = impute_missing_dates(df)

In [ ]:
df1 = pd.concat(dfs_dict.values())
df1

Let's now obtain the train and validation set. We are going to split the dataset into 2 sets, according to the building id.

In [ ]:
dfs_train, dfs_val, dfs_test = train_val_test_split(df1)
train = pd.concat(dfs_train.values())

In [ ]:
val = pd.concat(dfs_val.values())

In [ ]:
test = pd.concat(dfs_test.values())

In [ ]:
train

In [ ]:
val

In [ ]:
test

# Training the model

In [ ]:
train_window = 72

In [ ]:
X_train, y_train = create_multivariate_train_eval_sequences(train, train_window)

In [ ]:
X_train, y_train

In [ ]:
X_train.shape, y_train.shape

In [ ]:
BATCH_SIZE =  128
N_EPOCHS = 40
hidden_size = 1/8

In [ ]:
w_size = X_train.shape[1] * X_train.shape[2]
z_size = w_size * hidden_size 
w_size, z_size

In [ ]:
z_size = int(z_size)

In [ ]:
z_size

In [ ]:
import torch.utils.data as data_utils

In [ ]:
train_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(torch.from_numpy(X_train).float().contiguous().view(([X_train.shape[0], w_size]))), batch_size = BATCH_SIZE, shuffle = False, num_workers = 0) #.view(([X_train.shape[0], w_size]))

In [ ]:
X_val, y_val = create_multivariate_train_eval_sequences(val, train_window)

In [ ]:
X_val.shape, y_val.shape

In [ ]:
val_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(torch.from_numpy(X_val).float().contiguous().view(([X_val.shape[0],w_size]))) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0) #.view(([X_val.shape[0],w_size]))

In [ ]:
!export CUDA_VISIBLE_DEVICES=2

In [ ]:
device = get_default_device()
device

In [ ]:
model = UsadModel(w_size, z_size)
model = to_device(model,device)

In [ ]:
print(device)

In [ ]:
history = training(N_EPOCHS,model,train_loader,val_loader) #2.15 min a epoch

In [ ]:
plot_history(history)

In [ ]:
print(model)

In [ ]:
torch.save({
            'encoder': model.encoder.state_dict(),
            'decoder1': model.decoder1.state_dict(),
            'decoder2': model.decoder2.state_dict()
            }, "/home/medoro/Unsupervised_Anomaly_Detection_thesis/checkpoints/model_40epochs_multivariate.pth")

# Testing

In [ ]:
checkpoint = torch.load("/home/medoro/Unsupervised_Anomaly_Detection_thesis/checkpoints/model_40epochs_multivariate.pth")

model.encoder.load_state_dict(checkpoint['encoder'])
model.decoder1.load_state_dict(checkpoint['decoder1'])
model.decoder2.load_state_dict(checkpoint['decoder2'])

In [ ]:
X_test, y_test = create_multivariate_train_eval_sequences(test, train_window)

In [ ]:
X_test.shape, y_test.shape

In [ ]:
test_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
    torch.from_numpy(X_test).float().contiguous().view(([X_test.shape[0],w_size]))
) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

## Testing (anomaly score-based)

In [ ]:
results=testing(model,test_loader) #Prova con il test set

Let's create the dataset to perform predictions.

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
dfs_dict_1 = {}
for building_id, gdf in test.groupby("building_id"):
  gdf[['meter_reading', 'sea_level_pressure', 'weekday_x', 'weekday_y']]=scaler.fit_transform(gdf[['meter_reading', 'sea_level_pressure', 'weekday_x', 'weekday_y']])
  dfs_dict_1[building_id] = gdf[train_window:]
predicted_df = pd.concat(dfs_dict_1.values())

In [ ]:
lista = []
for el in results:
  for el2 in el:
    lista.append(el2.cpu().item())

In [ ]:
lista

In [ ]:
plt.hist(lista, bins=50)
plt.xlabel("Anomaly score")
plt.ylabel("No of samples")
plt.show()

In [ ]:
predicted_df['predictions'] = lista

In [ ]:
predicted_df

In [ ]:
predicted_df.predictions.min(), predicted_df.predictions.max()

In [ ]:
perc = 90
threshold = (np.percentile(predicted_df.predictions.values, perc))
threshold

In [ ]:
predicted_df['threshold'] = threshold

In [ ]:
predicted_df['predicted_anomaly'] = predicted_df.predictions > predicted_df['threshold']
predicted_df['predicted_anomaly']=predicted_df['predicted_anomaly'].replace(False,0)
predicted_df['predicted_anomaly']=predicted_df['predicted_anomaly'].replace(True,1)

In [ ]:
predicted_df

In [ ]:
predicted_df.predicted_anomaly.unique()

In [ ]:
len(predicted_df[predicted_df.predicted_anomaly == 1])/len(predicted_df)

In [ ]:
predicted_df.index.names=['timestamp']
predicted_df= predicted_df.reset_index()

In [ ]:
predicted_df = pd.merge(predicted_df, df[['timestamp','building_id']], on=['timestamp','building_id'])

In [ ]:
print(classification_report(predicted_df.anomaly, predicted_df.predicted_anomaly))

In [ ]:
roc_auc_score(predicted_df['anomaly'], predicted_df['predicted_anomaly'])

## Testing (reconstruction-based)

For this we are going to consider non-overlapping windows.

In [ ]:
X_test, y_test = create_multivariate_test_sequences(test, train_window)

In [ ]:
X_test.shape, y_test.shape

In [ ]:
test_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
    torch.from_numpy(X_test).float().contiguous().view(([X_test.shape[0],w_size]))
) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [ ]:
results, w1, w2 = testing_prova(model, test_loader)

In [ ]:
w1

In [ ]:
w2

In [ ]:
len(w1)

In [ ]:
len(w2)

In [ ]:
len(w1[1]), len(w1[-1])

In [ ]:
len(w2[1]), len(w2[-1])

In [ ]:
# Fai reshape da [batch, 360] a [batch, 72, 5]
reshaped_w1 = [torch.reshape(w1_el, (w1_el.size()[0], int(w1_el.size()[1]/5), int(w1_el.size()[1]/72))) for w1_el in w1]
reshaped_w1_try = [torch.reshape(w1_el, (w1_el.size()[0]*w1_el.size()[1], w1_el.size()[2])) for w1_el in reshaped_w1]

In [ ]:
len(reshaped_w1_try), reshaped_w1_try[0].size(), reshaped_w1_try[-1].size()

In [ ]:
stacked = torch.stack(reshaped_w1_try[:-1])
stacked.shape

In [ ]:
stacked_reshaped = torch.reshape(stacked, (stacked.size()[0] * stacked.size()[1], stacked.size()[2]))
stacked_reshaped.size()

In [ ]:
stacked_array = stacked_reshaped.cpu().numpy()
stacked_array.shape

In [ ]:
last_array = reshaped_w1_try[-1].cpu().numpy()
last_array.shape

In [ ]:
total1 = np.concatenate([stacked_array, last_array])
total1.shape

Let's do the same for w2.

In [ ]:
# Fai reshape da [batch, 360] a [batch, 72, 5]
reshaped_w2 = [torch.reshape(w2_el, (w2_el.size()[0], int(w2_el.size()[1]/5), int(w2_el.size()[1]/72))) for w2_el in w2]
reshaped_w2_try = [torch.reshape(w2_el, (w2_el.size()[0]*w2_el.size()[1], w2_el.size()[2])) for w2_el in reshaped_w2]

In [ ]:
stacked2 = torch.stack(reshaped_w2_try[:-1])
stacked2.shape

In [ ]:
stacked_reshaped2 = torch.reshape(stacked2, (stacked2.size()[0] * stacked2.size()[1], stacked2.size()[2]))
stacked_reshaped2.size()

In [ ]:
stacked_array = stacked_reshaped2.cpu().numpy()
stacked_array

In [ ]:
last_array2 = reshaped_w2_try[-1].cpu().numpy()
last_array2

In [ ]:
total2 = np.concatenate([stacked_array, last_array2])

In [ ]:
total2.shape

Let's now try to see how this refers to the reconstructed time series.

In [ ]:
w1_reco = pd.DataFrame(total1)
w1_reco

In [ ]:
w2_reco = pd.DataFrame(total2)
w2_reco

The first column is the one we are interested in, as it refers to the meter reading. Let's create the final dataset and concatenate the reconstructions to it.

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
dfs_dict_1 = {}
for building_id, gdf in test.groupby("building_id"):
  gdf[['meter_reading', 'sea_level_pressure', 'weekday_x', 'weekday_y']]=scaler.fit_transform(gdf[['meter_reading', 'sea_level_pressure', 'weekday_x', 'weekday_y']])
  dfs_dict_1[building_id] = gdf
predicted_df_test = pd.concat(dfs_dict_1.values())

In [ ]:
predicted_df_test['reconstruction'] = w1_reco.loc[:, 0].values

In [ ]:
predicted_df_test['reconstruction2'] = w2_reco.loc[:, 0].values

In [ ]:
predicted_df_test

In [ ]:
predicted_df_test.reconstruction2.min(), predicted_df_test.reconstruction2.max()

In [ ]:
predicted_df_test.reconstruction.min(), predicted_df_test.reconstruction.max()

In [ ]:
predicted_df_test.meter_reading.min(), predicted_df_test.meter_reading.max()

In [ ]:
predicted_df_test['relative_loss'] = np.abs((predicted_df_test['reconstruction']-predicted_df_test['meter_reading'])/predicted_df_test['reconstruction'])

In [ ]:
predicted_df_test['relative_loss2'] = np.abs((predicted_df_test['reconstruction2']-predicted_df_test['meter_reading'])/predicted_df_test['reconstruction2'])

In [ ]:
#calculate threshold on relative loss quartiles but only on val, and in this case per building
thresholds=np.array([])
for building_id, gdf in predicted_df_test.groupby("building_id"):
  val_mre_loss_building= gdf['relative_loss'].values
  building_threshold = (np.percentile(val_mre_loss_building, 75)) + 1.5 *((np.percentile(val_mre_loss_building, 75))-(np.percentile(val_mre_loss_building, 25)))
  gdf['threshold']=building_threshold
  thresholds= np.append(thresholds, gdf['threshold'].values)
print(thresholds.shape)
predicted_df_test['threshold']= thresholds

In [ ]:
#calculate threshold on relative loss quartiles but only on val, and in this case per building
thresholds=np.array([])
for building_id, gdf in predicted_df_test.groupby("building_id"):
  val_mre_loss_building= gdf['relative_loss2'].values
  building_threshold = (np.percentile(val_mre_loss_building, 75)) + 1.5 *((np.percentile(val_mre_loss_building, 75))-(np.percentile(val_mre_loss_building, 25)))
  gdf['threshold2']=building_threshold
  thresholds= np.append(thresholds, gdf['threshold2'].values)
print(thresholds.shape)
predicted_df_test['threshold2']= thresholds

In [ ]:
predicted_df_test

In [ ]:
predicted_df_test['predicted_anomaly'] = predicted_df_test['relative_loss'] > predicted_df_test['threshold']
predicted_df_test['predicted_anomaly']=predicted_df_test['predicted_anomaly'].replace(False,0)
predicted_df_test['predicted_anomaly']=predicted_df_test['predicted_anomaly'].replace(True,1)

In [ ]:
predicted_df_test['predicted_anomaly2'] = predicted_df_test['relative_loss2'] > predicted_df_test['threshold2']
predicted_df_test['predicted_anomaly2']=predicted_df_test['predicted_anomaly2'].replace(False,0)
predicted_df_test['predicted_anomaly2']=predicted_df_test['predicted_anomaly2'].replace(True,1)

In [ ]:
predicted_df_test.index.names=['timestamp']
predicted_df_test= predicted_df_test.reset_index()

In [ ]:
predicted_df_test.predicted_anomaly.unique()

In [ ]:
predicted_df_test.predicted_anomaly2.unique()

In [ ]:
predicted_anomalies = predicted_df_test.loc[predicted_df_test['predicted_anomaly'] == 1]
predicted_anomalies2 = predicted_df_test.loc[predicted_df_test['predicted_anomaly2'] == 1]
true_anomalies = predicted_df_test.loc[predicted_df_test['anomaly'] == 1]

In [ ]:
predicted_df_test = pd.merge(predicted_df_test, df[['timestamp','building_id']], on=['timestamp','building_id'])

In [ ]:
print(classification_report(predicted_df_test['anomaly'], predicted_df_test['predicted_anomaly']))

In [ ]:
print(classification_report(predicted_df_test['anomaly'], predicted_df_test['predicted_anomaly2']))

In [ ]:
roc_auc_score(predicted_df_test['anomaly'], predicted_df_test['predicted_anomaly'])

In [ ]:
roc_auc_score(predicted_df_test['anomaly'], predicted_df_test['predicted_anomaly2'])

In [ ]:
predicted_df_test.building_id.unique()

In [ ]:
visualizations = predicted_df_test[predicted_df_test.building_id == 1319]
visualizations

In [ ]:
plt.plot(visualizations.meter_reading, label = "meter reading") #predicted_df_test.meter_reading[:8784]
plt.plot(visualizations.reconstruction, label = "w1_reconstruction")
plt.plot(visualizations.reconstruction2, label = "w2_reconstruction")
plt.legend()
plt.show()

In [ ]:
predicted_anomalies1 = visualizations.loc[visualizations['predicted_anomaly'] == 1]
predicted_anomalies2 = visualizations.loc[visualizations['predicted_anomaly2'] == 1]
true_anomalies = visualizations.loc[visualizations['anomaly'] == 1]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=visualizations.index, y=visualizations['meter_reading'], name='meter readings'))
fig.add_trace(go.Scatter(x=visualizations.index, y=visualizations['reconstruction'], name='meter readings reconstructed'))
fig.add_trace(go.Scatter(x=visualizations.index, y=visualizations['reconstruction2'], name='meter readings reconstructed'))
fig.add_trace(go.Scatter(x=true_anomalies.index, y=true_anomalies['meter_reading'], mode='markers', marker=dict(color='forestgreen'), name='True_Anomaly'))
fig.add_trace(go.Scatter(x=predicted_anomalies1.index, y=predicted_anomalies1['meter_reading'], mode='markers', marker=dict(color='yellow'), name='True_Anomaly'))
fig.add_trace(go.Scatter(x=predicted_anomalies2.index, y=predicted_anomalies2['meter_reading'], mode='markers', marker=dict(color='orange'), name='True_Anomaly'))
fig.update_layout(showlegend=True, title='meter readings predicted and anomalies - val')
fig.show()